# Neo4j에 그래프 로드

In [17]:
import os
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

In [18]:
import pickle

with open('samples/graph_docs.pkl', 'rb') as f:
    loaded_graph_docs = pickle.load(f)

In [19]:
from langchain.graphs import Neo4jGraph
from py2neo import Graph

def init_graph(vector_search_enabled: bool):
    py_graph.run("MATCH (n) DETACH DELETE n")

    if vector_search_enabled:
        py_graph.run("""
        CREATE VECTOR INDEX content_embedding_index IF NOT EXISTS
        FOR (n:Content) ON (n.embedding)
        OPTIONS {
            indexConfig: {
                `vector.dimensions`: 1024,
                `vector.similarity_function`: 'cosine'
            }
        }
        """)

py_graph = Graph()
n4j_graph = Neo4jGraph()

vector_search_enabled = False
init_graph(vector_search_enabled)

In [20]:
n4j_graph.add_graph_documents(
  loaded_graph_docs, 
  baseEntityLabel=True
)

In [21]:
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import Neo4jVector

embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

vector_store = Neo4jVector.from_existing_index(
    embedding=embeddings,
    index_name="content_embedding_index",
    node_label="Content",
    text_node_property="text", 
    embedding_node_property="embedding"
)

In [22]:
if vector_search_enabled:
    query = "No Code RAG"
    results = vector_store.similarity_search(query, k=10)
    for result in results:
        print("=================\n")
        print(result.page_content)
        print("=================\n")